In [1]:
required_packages = {"keplergl","geopandas","h3","geocoder"}

import pkg_resources
for lib in required_packages - {pkg.key for pkg in pkg_resources.working_set}:
  print(f"installing {lib}")
  %pip install -q --upgrade pip
  %pip install --use-feature=2020-resolver -q $lib


installing geocoder
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
WARNING: --use-feature=2020-resolver no longer has any effect, since it is now the default dependency resolver in pip. This will become an error in pip 21.0.
Python interpreter will be restarted.
installing h3
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
WARNING: --use-feature=2020-resolver no longer has any effect, since it is now the default dependency resolver in pip. This will become an error in pip 21.0.
Python interpreter will be restarted.
installing keplergl
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
WARNING: --use-feature=2020-resolver no longer has any effect, since it is now the default dependency resolver in pip. This will become an error in pip 21.0.
Python interpreter will be restarted.
installing geopandas
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
WARNING: --use-feature=2020-resolver no longer has any effect, since it is now the default dependency resolver in pip. This will become an error in pip 21.0.
Python interpreter will be restarted.

In [2]:
%python

# https://databricks.com/notebooks/point-in-polygon-with-h3_gtags.html

import keplergl
import tempfile

if not 'orig_repr_html_' in locals():
  orig_repr_html_ = keplergl.KeplerGl._repr_html_

def html_patch(self):
  """This is the patch to make it work well in Databricks"""
  # temp file purposely doesn't get cleaned up, but should if that's desired
  (_, tmp) = tempfile.mkstemp() 
  frame_bytes = orig_repr_html_(self)
  frame_str = str(frame_bytes, encoding='utf-8')
  # This additional script for fixing the height is necessary because kepler doesn't embed well, and
  # mutates containing document directly. The height parameter to keplergl.KeplerGl will be the initial 
  # height of the result iframe. 
  return f"""{frame_str}<script>
    var targetHeight = "{self.height or 600}px";
    var interval = window.setInterval(function() {{
      if (document.body && document.body.style && document.body.style.height !== targetHeight) {{
        document.body.style.height = targetHeight;
      }}
    }}, 250);</script>""";
#setattr(keplergl.KeplerGl, '_repr_html_', html_patch)

In [3]:
import keplergl
map_1 = keplergl.KeplerGl(height=600)
keplergl_html = str(map_1._repr_html_(),'utf-8')

import base64
b64 = base64.b64encode(keplergl_html.encode('utf-8'))
src = f"data:text/html;base64,{b64.decode('utf-8')}"
html = f'<iframe src="{src}" style="width:100%; height: 600px">'
displayHTML(html)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter